# Interactive Corporate Credit Report Generator (JSON-LD Focused)

This notebook allows you to input key company information. It will then populate a **master JSON-LD prompt template** with your data and simulated system-generated values. This populated JSON-LD prompt is displayed (and could be saved). Finally, it generates a *simulated* human-readable credit report in Markdown by interpreting the populated JSON-LD.

**Instructions:**
1. Fill in the input fields below.
2. Click the "Generate Populated JSON-LD & Simulated Report" button.
3. Review the generated populated JSON-LD prompt and the simulated Markdown report.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, Javascript
import json
import datetime
import re

# --- Master JSON-LD Prompt Template (Your Provided Structure) ---
MASTER_JSON_LD_TEMPLATE_STR = '''
{
  "@context": {
    "report": "https://ontology.yourcorp.com/creditreport/v1#",
    "cacm": "https://ontology.yourcorp.com/cacm/v1#",
    "credit": "https://ontology.yourcorp.com/credit_analysis/v2.1#",
    "schema": "http://schema.org/",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "dcterms": "http://purl.org/dc/terms/",
    "label": "rdfs:label",
    "description": "dcterms:description",
    "value": "schema:value",
    "unit": "schema:unitText",
    "trend": "report:trendAnalysis",
    "commentary": "report:commentary",
    "rationale": "report:rationale",
    "drivers": "report:keyDrivers",
    "triggers": "report:downgradeTriggers"
  },
  "@id": "urn:template:ComprehensiveCreditReport:v1.0",
  "@type": "report:CreditReportPromptTemplate",
  "templateVersion": "1.0",
  "templateDescription": "Comprehensive prompt template for generating a full corporate credit rating report.",
  "reportGenerationInstructions": {
    "@type": "report:GenerationInstructions",
    "overallTone": "Professional, analytical, objective, cautious, balanced.",
    "languageStyle": "Clear, concise, data-driven. Avoid jargon where possible or explain.",
    "guidanceOnMissingData": "If specific data for a section is NOT provided by user inputs or retrievable by CACMs, explicitly state 'Information not provided/available for this section.' or 'Analysis for this section is limited due to lack of specific input.' Do NOT invent data.",
    "outputFormatInstructions": "Primary output should be structured JSON-LD. Human-readable sections should be suitable for Markdown rendering.",
    "mandatoryDisclaimer": "This report is a simulated analysis based on user-provided and system-retrieved inputs. It should not be used for actual investment or credit decisions without independent verification by qualified professionals."
  },
  "targetCompanyInformation": {
    "@type": "report:CompanyInput",
    "companyName": "[USER_INPUT_CompanyName]",
    "tickerSymbol": "[USER_INPUT_TickerSymbol]",
    "primarySector": "[USER_INPUT_PrimarySector]",
    "primaryIndustry": "[USER_INPUT_PrimaryIndustry]",
    "countryOfDomicile": "[USER_INPUT_CountryOfDomicile]",
    "analystKeyAssumptions": "[USER_INPUT_KeyAssumptions]"
  },
  "reportSections": {
    "companyOverview": {
      "@type": "report:CompanyOverviewSection",
      "label": "Company Overview",
      "businessDescription": "[USER_INPUT_BusinessDescription OR LLM_CACM_Generate_BusinessDescription]",
      "productsAndServices": "[LLM_CACM_Describe_ProductsServices]",
      "operatingGeographies": "[LLM_CACM_Describe_OperatingGeographies]",
      "scaleAndMarketPosition": "[LLM_CACM_Assess_ScaleMarketPosition]",
      "corporateStructure": "[LLM_CACM_Describe_CorporateStructure]"
    },
    "recentDevelopments": {
      "@type": "report:RecentDevelopmentsSection",
      "label": "Recent Developments",
      "summaryOfRecentEvents": "[USER_INPUT_RecentNewsSnippets OR LLM_CACM_SummarizeNews]",
      "earningsSummary": "[LLM_CACM_SummarizeLatestEarnings]",
      "maActivity": "[LLM_CACM_SummarizeMA_Activity]",
      "financingActivities": "[LLM_CACM_SummarizeFinancingActivities]",
      "managementChanges": "[LLM_CACM_SummarizeManagementChanges]",
      "legalRegulatoryUpdates": "[LLM_CACM_SummarizeLegalRegulatory]",
      "potentialCreditImpactAnalysis": "[LLM_Analyze_CreditImpact_Of_Events]"
    },
    "industryAnalysisAndOutlook": {
      "@type": "report:IndustryAnalysisSection",
      "label": "Industry Analysis and Outlook",
      "industryDescription": "[LLM_CACM_Describe_Industry]",
      "industryTAM": "[USER_INPUT_TAM OR LLM_CACM_Estimate_TAM]",
      "industryGrowthOutlook": "[USER_INPUT_IndustryGrowthOutlook OR LLM_CACM_Assess_IndustryGrowth]",
      "keyIndustryTrends": "[LLM_CACM_Identify_IndustryTrends]",
      "industryRiskFactors": "[LLM_CACM_Identify_IndustryRisks]",
      "competitiveLandscape": "[USER_INPUT_CompetitiveLandscape OR LLM_CACM_Analyze_Competition]",
      "companyPositioning": "[LLM_CACM_Assess_CompanyPosition_In_Industry]"
    },
    "managementAndGovernance": {
      "@type": "report:ManagementGovernanceSection",
      "label": "Management & Governance Assessment",
      "managementStrategyOverview": "[USER_INPUT_ManagementStrategy OR LLM_CACM_Assess_ManagementStrategy]",
      "managementTrackRecord": "[LLM_CACM_Assess_ManagementTrackRecord]",
      "financialPolicy": "[LLM_CACM_Assess_FinancialPolicy]",
      "riskManagementFramework": "[LLM_CACM_Assess_RiskManagement]",
      "boardStructureAndEffectiveness": "[LLM_CACM_Assess_Board]",
      "shareholderStructure": "[LLM_CACM_Describe_Shareholders]",
      "governanceConcernsOrStrengths": "[LLM_Identify_GovernanceIssues]"
    },
    "esgFactors": {
      "@type": "report:ESGSection",
      "label": "Environmental, Social, and Governance (ESG) Factors",
      "environmentalSummary": "[USER_INPUT_ESG_Environmental OR LLM_CACM_Assess_ESG_E]",
      "socialSummary": "[USER_INPUT_ESG_Social OR LLM_CACM_Assess_ESG_S]",
      "governanceSummary": "[USER_INPUT_ESG_Governance OR LLM_CACM_Assess_ESG_G]",
      "overallESGCreditImpact": "[LLM_Assess_Overall_ESG_CreditImpact]"
    },
    "financialPerformanceAnalysis": {
      "@type": "report:FinancialAnalysisSection",
      "label": "Financial Performance Analysis",
      "analysisPeriod": "[USER_INPUT_FinancialAnalysisPeriod]",
      "dataSourceNotes": "[LLM_CACM_Note_FinancialDataSources_Adjustments]",
      "revenueAnalysis": {
        "totalRevenue": {"value": "[USER_INPUT_Revenue_CurrentYear]", "unit": "USD_M"},
        "revenueGrowthYoY": {"value": "[USER_INPUT_RevenueGrowth_YoY_Percent]", "unit": "%"},
        "revenueTrend": "[LLM_Analyze_RevenueTrend_3-5Years]",
        "revenueDrivers": "[LLM_Identify_RevenueDrivers]",
        "revenueCommentary": "[LLM_CACM_Provide_RevenueCommentary]"
      },
      "profitabilityAnalysis": {
        "ebitda": {"value": "[USER_INPUT_EBITDA_CurrentYear]", "unit": "USD_M"},
        "ebitdaMargin": {"value": "[USER_INPUT_EBITDA_Margin_Percent]", "unit": "%"},
        "ebitdaTrend": "[LLM_Analyze_EBITDATrend_3-5Years]",
        "ebitdaMarginTrend": "[LLM_Analyze_EBITDAMarginTrend_3-5Years]",
        "profitabilityDrivers": "[LLM_Identify_ProfitabilityDrivers]",
        "profitabilityCommentary": "[LLM_CACM_Provide_ProfitabilityCommentary]"
      },
      "cashFlowAnalysis": {
        "operatingCashFlow": {"value": "[USER_INPUT_OCF_CurrentYear]", "unit": "USD_M"},
        "capitalExpenditures": {"value": "[USER_INPUT_Capex_CurrentYear]", "unit": "USD_M"},
        "freeCashFlow": {"value": "[USER_INPUT_FCF_CurrentYear]", "unit": "USD_M"},
        "fcfTrend": "[LLM_Analyze_FCFTrend_3-5Years]",
        "fcfConversionRate": {"value": "[CALCULATED_FCF_EBITDA_Percent]", "unit": "%"},
        "cashFlowDrivers": "[LLM_Identify_CashFlowDrivers]",
        "cashFlowCommentary": "[LLM_CACM_Provide_CashFlowCommentary]"
      },
      "leverageAnalysis": {
        "grossDebt": {"value": "[USER_INPUT_GrossDebt_Current]", "unit": "USD_M"},
        "cashAndEquivalents": {"value": "[USER_INPUT_Cash_Current]", "unit": "USD_M"},
        "netDebt": {"value": "[CALCULATED_NetDebt]", "unit": "USD_M"},
        "grossDebtToEbitda": {"value": "[USER_INPUT_Debt_EBITDA_Ratio]", "unit": "x"},
        "leverageTrend": "[LLM_Analyze_LeverageTrend_3-5Years]",
        "leverageDrivers": "[LLM_Identify_LeverageDrivers]",
        "debtStructureMaturityProfile": "[LLM_CACM_Analyze_DebtStructure]",
        "leverageCommentary": "[LLM_CACM_Provide_LeverageCommentary]"
      },
      "coverageAnalysis": {
        "ebitdaToInterestExpense": {"value": "[USER_INPUT_EBITDA_Interest_Ratio]", "unit": "x"},
        "fcfToDebt": {"value": "[USER_INPUT_FCF_Debt_Ratio_Percent OR CALCULATED]", "unit": "%"},
        "coverageTrend": "[LLM_Analyze_CoverageTrend_3-5Years]",
        "coverageCommentary": "[LLM_CACM_Provide_CoverageCommentary]"
      },
      "covenantCompliance": {
        "keyCovenants": "[USER_INPUT_CovenantDetails OR LLM_CACM_Identify_Covenants]",
        "complianceStatus": "[USER_INPUT_CovenantComplianceStatus OR LLM_Assess_CovenantHeadroom]",
        "covenantCommentary": "[LLM_CACM_Provide_CovenantCommentary]"
      },
      "liquidityAnalysis": {
        "currentCashPosition": {"value": "[USER_INPUT_Cash_Current]", "unit": "USD_M"},
        "undrawnRevolverAvailability": {"value": "[USER_INPUT_RevolverAvailability]", "unit": "USD_M"},
        "nearTermDebtMaturities": "[LLM_CACM_List_NearTermMaturities]",
        "workingCapitalAnalysis": "[LLM_CACM_Analyze_WorkingCapital]",
        "liquiditySourcesVsUses": "[LLM_CACM_Assess_LiquiditySourcesUses]",
        "liquidityCommentary": "[LLM_CACM_Provide_LiquidityCommentary]"
      },
      "performanceVsBudgetProjections": {
        "varianceAnalysisRevenue": "[USER_INPUT_BudgetVariance_Revenue OR LLM_Compare_ActualVsBudget]",
        "varianceAnalysisEbitda": "[USER_INPUT_BudgetVariance_EBITDA OR LLM_Compare_ActualVsBudget]",
        "varianceAnalysisFcf": "[USER_INPUT_BudgetVariance_FCF OR LLM_Compare_ActualVsBudget]",
        "reasonsForVariance": "[USER_INPUT_ReasonsForVariance OR LLM_Explain_Variance]",
        "performanceCommentary": "[LLM_CACM_Provide_BudgetPerformanceCommentary]"
      }
    },
    "peerAnalysis": {
      "@type": "report:PeerAnalysisSection",
      "label": "Peer Analysis",
      "selectedPeerGroup": "[USER_INPUT_PeerGroupList OR LLM_CACM_Identify_PeerGroup]",
      "comparativeFinancialMetrics": "[LLM_CACM_Generate_PeerFinancialTable]",
      "comparativeValuationMetrics": "[LLM_CACM_Generate_PeerValuationTable]",
      "peerAnalysisCommentary": "[LLM_CACM_Provide_PeerAnalysisCommentary]"
    },
    "dcfValuationAndDebtRepaymentAnalysis": {
      "@type": "report:ValuationAnalysisSection",
      "label": "DCF, Valuation & Debt Repayment Capacity",
      "dcfAnalysis": {
        "forecastPeriodYears": "[USER_INPUT_DCF_ForecastYears OR 7]",
        "wacc": {
          "calculatedValue": {"value": "[USER_INPUT_WACC_Percent OR LLM_CACM_Calculate_WACC]", "unit": "%"},
          "keyAssumptions": "[LLM_CACM_List_WACC_Assumptions]"
        },
        "terminalGrowthRate": {"value": "[USER_INPUT_TerminalGrowth_Percent OR LLM_Estimate_TGR]", "unit": "%"},
        "projectedFreeCashFlows": "[LLM_CACM_Project_FCF_Table]",
        "calculatedEnterpriseValue": {"value": "[LLM_CACM_Calculate_EV_from_DCF]", "unit": "USD_M"},
        "calculatedEquityValue": {"value": "[LLM_CACM_Calculate_EquityValue_from_EV]", "unit": "USD_M"},
        "dcfCommentaryAndSensitivity": "[LLM_Provide_DCF_Commentary_Sensitivity]"
      },
      "debtRepaymentCapacity": {
        "analysisType": "Cumulative FCF to Debt",
        "analysisPeriodYears": "[USER_INPUT_DebtRepayment_ForecastYears OR 7]",
        "cumulativeFcf": {"value": "[CALCULATED_CumulativeFCF_OverPeriod]", "unit": "USD_M"},
        "totalDebtForRepayment": {"value": "[USER_INPUT_TotalDebt_Current OR ProformaDebt]", "unit": "USD_M"},
        "repaymentCapacityRatio": {"value": "[CALCULATED_CumulativeFCF_to_Debt_Percent]", "unit": "%"},
        "debtRepaymentCommentary": "[LLM_CACM_Assess_DebtRepaymentCapacity]"
      }
    },
    "creditRatingAssessment": {
      "@type": "report:CreditRatingSection",
      "label": "Credit Rating Assessment",
      "internalCreditRating": {
        "ratingValue": "[LLM_Determine_InternalCreditRating]",
        "ratingScale": "Internal Corporate Rating Scale",
        "ratingOutlook": "[LLM_Determine_RatingOutlook]",
        "ratingAssignedDate": "[CURRENT_DATE]",
        "ratingRationale": "[LLM_CACM_Generate_DetailedRatingRationale]",
        "keyPositiveRatingFactors": "[LLM_List_PositiveRatingFactors]",
        "keyNegativeRatingFactors": "[LLM_List_NegativeRatingFactors]"
      },
      "sharedNationalCreditSNC": {
        "regulatoryRating": "[USER_INPUT_SNC_Rating OR LLM_Retrieve_SNC_IfAvailable]",
        "ratingDate": "[USER_INPUT_SNC_Date]",
        "sncRationale": "[USER_INPUT_SNC_Rationale OR LLM_Summarize_SNC_Rationale]",
        "sncCommentary": "[LLM_Provide_SNC_Context_Commentary]"
      },
      "keyDowngradeTriggers": "[LLM_Identify_KeyDowngradeTriggers]",
      "keyUpgradeConsiderations": "[LLM_Identify_KeyUpgradeConsiderations]"
    },
    "summaryStrengthsAndWeaknesses": {
      "@type": "report:SWSection",
      "label": "Summary: Key Credit Strengths and Weaknesses",
      "keyStrengths": [
        {"strength": "[LLM_Synthesize_Strength_1]", "supportingEvidence": "[Brief evidence]"},
        {"strength": "[LLM_Synthesize_Strength_2]", "supportingEvidence": "[Brief evidence]"},
        {"strength": "[LLM_Synthesize_Strength_3]", "supportingEvidence": "[Brief evidence]"}
      ],
      "keyWeaknesses": [
        {"weakness": "[LLM_Synthesize_Weakness_1]", "mitigantsOrContext": "[Brief context]"},
        {"weakness": "[LLM_Synthesize_Weakness_2]", "mitigantsOrContext": "[Brief context]"},
        {"weakness": "[LLM_Synthesize_Weakness_3]", "mitigantsOrContext": "[Brief context]"}
      ]
    },
    "appendix": {
      "@type": "report:AppendixSection",
      "label": "Appendix",
      "detailedFinancialStatements": "[Placeholder_Link_To_Financials OR LLM_Generate_Summary_Tables]",
      "peerGroupDefinition": "[Placeholder_PeerList_Criteria]",
      "ratingScaleDefinitions": "[Placeholder_InternalRatingScaleDescription]"
    }
  }
}
'''
MASTER_JSON_LD_TEMPLATE = json.loads(MASTER_JSON_LD_TEMPLATE_STR)

# --- Report Structure Guide for Markdown (Simpler than full JSON-LD for direct LLM generation of this part) ---
REPORT_STRUCTURE_GUIDE_MD = '''
# Corporate Credit Report: {company_name} ({company_ticker})
Report Date: {report_date}
Analyst: [USER_NAME_HERE / AI Assistant]

## 1. Executive Summary
   * **Overall Assessment:** {exec_summary_overall_assessment}
   * **Simulated Internal Credit Rating:** {exec_summary_simulated_rating}
   * **Rating Outlook:** {exec_summary_rating_outlook}
   * **Key Positive Factors:** 
{exec_summary_key_positive_factors}
   * **Key Credit Concerns:** 
{exec_summary_key_credit_concerns}

## 2. Company Overview
   * **Company Name:** {company_name}
   * **Ticker Symbol:** {company_ticker}
   * **Primary Sector:** {company_sector}
   * **Primary Industry:** {company_industry}
   * **Country of Domicile:** {company_country}
   * **Brief Business Description:** {company_business_description}
   * **Products & Services:** {company_products_services}
   * **Operating Geographies:** {company_operating_geographies}
   * **Scale & Market Position:** {company_scale_market_position}
   * **Corporate Structure:** {company_corporate_structure}

## 3. Key Analyst Assumptions
{analyst_key_assumptions}

## 4. Recent Developments
   * **Summary of Recent Events:** {recent_dev_summary}
   * **Latest Earnings Summary:** {recent_dev_earnings}
   * **M&A Activity:** {recent_dev_ma}
   * **Financing Activities:** {recent_dev_financing}
   * **Management Changes:** {recent_dev_mgmt_changes}
   * **Legal/Regulatory Updates:** {recent_dev_legal}
   * **Potential Credit Impact Analysis:** {recent_dev_impact}

## 5. Industry Analysis and Outlook
   * **Industry Description:** {industry_description}
   * **Industry TAM:** {industry_tam}
   * **Industry Growth Outlook:** {industry_growth_outlook}
   * **Key Industry Trends:** {industry_key_trends}
   * **Industry Risk Factors:** {industry_risk_factors}
   * **Competitive Landscape:** {industry_competitive_landscape}
   * **Company Positioning in Industry:** {industry_company_positioning}

## 6. Management & Governance Assessment
   * **Management Strategy Overview:** {mgm_gov_strategy}
   * **Management Track Record:** {mgm_gov_track_record}
   * **Financial Policy:** {mgm_gov_financial_policy}
   * **Risk Management Framework:** {mgm_gov_risk_mgmt}
   * **Board Structure & Effectiveness:** {mgm_gov_board}
   * **Shareholder Structure:** {mgm_gov_shareholders}
   * **Governance Concerns/Strengths:** {mgm_gov_issues}

## 7. ESG Factors
   * **Environmental Summary:** {esg_environmental}
   * **Social Summary:** {esg_social}
   * **Governance Summary:** {esg_governance}
   * **Overall ESG Credit Impact:** {esg_overall_impact}

## 8. Financial Performance Analysis (Period: {financial_analysis_period})
   * **Data Sources & Adjustments:** {financial_data_sources_notes}
   * **Revenue Analysis:**
       * Total Revenue: {financial_revenue_total} ({financial_revenue_growth_yoy}% YoY)
       * Trend: {financial_revenue_trend}
       * Drivers: {financial_revenue_drivers}
       * Commentary: {financial_revenue_commentary}
   * **Profitability Analysis:**
       * EBITDA: {financial_ebitda_total} (Margin: {financial_ebitda_margin}%)
       * EBITDA Trend: {financial_ebitda_trend}
       * EBITDA Margin Trend: {financial_ebitda_margin_trend}
       * Drivers: {financial_profitability_drivers}
       * Commentary: {financial_profitability_commentary}
   * **Cash Flow Analysis:**
       * Operating Cash Flow: {financial_ocf}
       * Capital Expenditures: {financial_capex}
       * Free Cash Flow (OCF - Capex): {financial_fcf}
       * FCF Trend: {financial_fcf_trend}
       * FCF Conversion (FCF/EBITDA): {financial_fcf_conversion}%
       * Drivers: {financial_cashflow_drivers}
       * Commentary: {financial_cashflow_commentary}
   * **Leverage Analysis:**
       * Gross Debt: {financial_gross_debt}
       * Cash & Equivalents: {financial_cash}
       * Net Debt: {financial_net_debt}
       * Gross Debt / EBITDA: {financial_debt_ebitda_ratio}x
       * Trend: {financial_leverage_trend}
       * Drivers: {financial_leverage_drivers}
       * Debt Structure & Maturity Profile: {financial_debt_structure}
       * Commentary: {financial_leverage_commentary}
   * **Coverage Analysis:**
       * EBITDA / Interest Expense: {financial_ebitda_interest_ratio}x
       * FCF / Debt: {financial_fcf_debt_ratio}%
       * Trend: {financial_coverage_trend}
       * Commentary: {financial_coverage_commentary}
   * **Covenant Compliance:**
       * Key Covenants: {financial_covenants_details}
       * Compliance Status & Headroom: {financial_covenants_status}
       * Commentary: {financial_covenants_commentary}
   * **Liquidity Analysis:**
       * Current Cash Position: {financial_liquidity_cash}
       * Undrawn Revolver Availability: {financial_liquidity_revolver}
       * Near-Term Debt Maturities (12-24m): {financial_liquidity_maturities}
       * Working Capital Analysis: {financial_liquidity_wc}
       * Sources vs. Uses Commentary: {financial_liquidity_sources_uses}
       * Overall Liquidity Commentary: {financial_liquidity_commentary}
   * **Performance vs. Budget/Projections (if available):**
       * Revenue Variance: {financial_variance_revenue}
       * EBITDA Variance: {financial_variance_ebitda}
       * FCF Variance: {financial_variance_fcf}
       * Reasons for Variance: {financial_variance_reasons}
       * Commentary: {financial_variance_commentary}

## 9. Peer Analysis
   * **Selected Peer Group:** {peer_group_list}
   * **Comparative Financial Metrics:** 
{peer_financial_table}
   * **Comparative Valuation Metrics:** 
{peer_valuation_table}
   * **Commentary:** {peer_analysis_commentary}

## 10. DCF Valuation & Debt Repayment Capacity (Illustrative)
   * **DCF Analysis:**
       * Forecast Period: {dcf_forecast_years} years
       * WACC: {dcf_wacc}% (Key Assumptions: {dcf_wacc_assumptions})
       * Terminal Growth Rate: {dcf_terminal_growth_rate}%
       * Projected FCFs: {dcf_projected_fcfs_table}
       * Calculated Enterprise Value (EV): {dcf_ev}
       * Calculated Equity Value: {dcf_equity_value}
       * Commentary & Sensitivity: {dcf_commentary_sensitivity}
   * **Debt Repayment Capacity (Cumulative FCF to Debt):
       * Analysis Period: {debt_repayment_years} years
       * Cumulative FCF: {debt_repayment_cumulative_fcf}
       * Total Debt for Repayment: {debt_repayment_total_debt}
       * Repayment Capacity Ratio: {debt_repayment_capacity_ratio}%
       * Commentary: {debt_repayment_commentary}

## 11. Credit Rating Assessment
   * **Simulated Internal Credit Rating:** {rating_internal_value}
   * **Rating Scale:** {rating_internal_scale}
   * **Rating Outlook:** {rating_internal_outlook}
   * **Date Assigned:** {rating_date}
   * **Rating Rationale:** {rating_rationale_detailed}
   * **Key Positive Rating Factors:** 
{rating_positive_factors}
   * **Key Negative Rating Factors:** 
{rating_negative_factors}
   * **Shared National Credit (SNC) Rating (if applicable):**
       * Regulatory Rating: {snc_rating_value}
       * Rating Date: {snc_rating_date}
       * Rationale: {snc_rationale}
       * Commentary: {snc_commentary}
   * **Key Downgrade Triggers:** 
{rating_downgrade_triggers}
   * **Key Upgrade Considerations:** 
{rating_upgrade_considerations}

## 12. Summary: Key Credit Strengths & Weaknesses
   * **Key Strengths:**
{summary_strengths_list}
   * **Key Weaknesses:**
{summary_weaknesses_list}

## 13. Appendix
   * **Detailed Financial Statements:** {appendix_financial_statements}
   * **Peer Group Definition:** {appendix_peer_definition}
   * **Rating Scale Definitions:** {appendix_rating_scale_defs}

## 14. Disclaimer
{report_disclaimer}
'''

# --- UI Configuration (Simplified for this example) ---
INPUT_UI_CONFIG = {
    "sections": [
        {
            "title": "Target Company Information",
            "fields": [
                {"name": "USER_INPUT_CompanyName", "label": "Company Name:", "type": "text", "default": "Acme Corp"},
                {"name": "USER_INPUT_TickerSymbol", "label": "Ticker Symbol:", "type": "text", "default": "ACME"},
                {"name": "USER_INPUT_PrimarySector", "label": "Primary Sector:", "type": "text", "default": "Technology"},
                {"name": "USER_INPUT_PrimaryIndustry", "label": "Primary Industry:", "type": "text", "default": "Software"},
                {"name": "USER_INPUT_CountryOfDomicile", "label": "Country of Domicile:", "type": "text", "default": "USA"},
                {"name": "USER_INPUT_KeyAssumptions", "label": "Analyst Key Assumptions:", "type": "textarea", "default": "1. Revenue growth of 5% YoY.\n2. Stable EBITDA margins around 20%.\n3. No major unannounced M&A."}
            ]
        },
        {
            "title": "Company Overview Inputs",
            "fields": [
                {"name": "USER_INPUT_BusinessDescription", "label": "Business Description:", "type": "textarea", "default": "Acme Corp is a leading provider of innovative widget solutions and services for the B2B market."}
            ]
        },
        {
            "title": "Recent Developments Inputs",
            "fields": [
                {"name": "USER_INPUT_RecentNewsSnippets", "label": "Recent News/Events:", "type": "textarea", "default": "- Q1 earnings beat expectations. - Announced strategic partnership with Beta Inc."}
            ]
        },
        {
            "title": "Industry Inputs",
            "fields": [
                {"name": "USER_INPUT_TAM", "label": "Industry TAM (e.g., $100B):", "type": "text", "default": "$50B"},
                {"name": "USER_INPUT_IndustryGrowthOutlook", "label": "Industry Growth Outlook:", "type": "textarea", "default": "Moderate growth expected (3-5%) driven by digitalization trends."},
                {"name": "USER_INPUT_CompetitiveLandscape", "label": "Competitive Landscape:", "type": "textarea", "default": "Highly competitive with 2 major players and several smaller niche providers."}
            ]
        },
        {
            "title": "Management & ESG Inputs",
            "fields": [
                {"name": "USER_INPUT_ManagementStrategy", "label": "Management Strategy Overview:", "type": "textarea", "default": "Focus on core product innovation and expanding into adjacent markets."},
                {"name": "USER_INPUT_ESG_Environmental", "label": "ESG - Environmental Notes:", "type": "textarea", "default": "Committed to carbon neutrality by 2030."},
                {"name": "USER_INPUT_ESG_Social", "label": "ESG - Social Notes:", "type": "textarea", "default": "Strong focus on employee development and DEI initiatives."},
                {"name": "USER_INPUT_ESG_Governance", "label": "ESG - Governance Notes:", "type": "textarea", "default": "Independent board majority, standard governance practices."}
            ]
        },
        {
            "title": "Financial Data Inputs (Annual - Current Year or TTM)",
            "fields": [
                {"name": "USER_INPUT_FinancialAnalysisPeriod", "label": "Analysis Period (e.g., FY2024):", "type": "text", "default": "TTM Q1 2025"},
                {"name": "USER_INPUT_Revenue_CurrentYear", "label": "Total Revenue ($M):", "type": "float", "default": 1200.0},
                {"name": "USER_INPUT_RevenueGrowth_YoY_Percent", "label": "Revenue Growth YoY (%):", "type": "float", "default": 5.5},
                {"name": "USER_INPUT_EBITDA_CurrentYear", "label": "EBITDA ($M):", "type": "float", "default": 240.0},
                {"name": "USER_INPUT_EBITDA_Margin_Percent", "label": "EBITDA Margin (%):", "type": "float", "default": 20.0},
                {"name": "USER_INPUT_OCF_CurrentYear", "label": "Operating Cash Flow ($M):", "type": "float", "default": 180.0},
                {"name": "USER_INPUT_Capex_CurrentYear", "label": "Capital Expenditures ($M):", "type": "float", "default": 60.0},
                {"name": "USER_INPUT_FCF_CurrentYear", "label": "Free Cash Flow ($M):", "type": "float", "default": 120.0},
                {"name": "USER_INPUT_GrossDebt_Current", "label": "Gross Debt ($M):", "type": "float", "default": 480.0},
                {"name": "USER_INPUT_Cash_Current", "label": "Cash & Equivalents ($M):", "type": "float", "default": 100.0},
                {"name": "USER_INPUT_Debt_EBITDA_Ratio", "label": "Gross Debt / EBITDA (x):", "type": "float", "default": 2.0},
                {"name": "USER_INPUT_EBITDA_Interest_Ratio", "label": "EBITDA / Interest Expense (x):", "type": "float", "default": 8.0},
                {"name": "USER_INPUT_FCF_Debt_Ratio_Percent", "label": "FCF / Debt (%):", "type": "float", "default": 25.0}
            ]
        },
        {
            "title": "Covenant & Liquidity Inputs",
            "fields": [
                {"name": "USER_INPUT_CovenantDetails", "label": "Key Covenant Details:", "type": "textarea", "default": "Max Debt/EBITDA < 3.5x; Min Interest Coverage > 2.0x"},
                {"name": "USER_INPUT_CovenantComplianceStatus", "label": "Covenant Compliance Status:", "type": "text", "default": "Currently in compliance with ample headroom."},
                {"name": "USER_INPUT_RevolverAvailability", "label": "Undrawn Revolver ($M):", "type": "float", "default": 150.0}
            ]
        },
        {
            "title": "Peer & Valuation Inputs",
            "fields": [
                {"name": "USER_INPUT_PeerGroupList", "label": "Peer Group (comma separated tickers):", "type": "text", "default": "PEER1, PEER2, PEER3"},
                {"name": "USER_INPUT_DCF_ForecastYears", "label": "DCF Forecast Years:", "type": "int", "default": 7},
                {"name": "USER_INPUT_WACC_Percent", "label": "WACC (%):", "type": "float", "default": 8.5},
                {"name": "USER_INPUT_TerminalGrowth_Percent", "label": "Terminal Growth Rate (%):", "type": "float", "default": 2.5},
                {"name": "USER_INPUT_DebtRepayment_ForecastYears", "label": "Debt Repayment Forecast Years:", "type": "int", "default": 7}
            ]
        },
        {
            "title": "SNC Rating Inputs (Optional)",
            "fields": [
                {"name": "USER_INPUT_SNC_Rating", "label": "SNC Regulatory Rating:", "type": "text", "default": "Pass"},
                {"name": "USER_INPUT_SNC_Date", "label": "SNC Rating Date (YYYY-MM-DD):", "type": "text", "default": "2024-10-15"},
                {"name": "USER_INPUT_SNC_Rationale", "label": "SNC Rationale Summary:", "type": "textarea", "default": "Stable performance and adequate capital levels."}
            ]
        }
    ]
}

# --- Dynamic Report Generation Logic ---
class ReportGeneratorSimulator:
    def __init__(self, master_json_ld_template, markdown_report_template):
        self.master_template = master_json_ld_template
        self.markdown_template = markdown_report_template

    def _get_with_default(self, data_dict, key, default='Not provided'):
        val = data_dict.get(key)
        if val is None or (isinstance(val, str) and not val.strip()):
            return default
        return val

    def _populate_json_ld_recursively(self, template_obj, user_inputs):
        if isinstance(template_obj, dict):
            new_dict = {}
            for k, v in template_obj.items():
                new_dict[k] = self._populate_json_ld_recursively(v, user_inputs)
            return new_dict
        elif isinstance(template_obj, list):
            return [self._populate_json_ld_recursively(item, user_inputs) for item in template_obj]
        elif isinstance(template_obj, str):
            # Handle USER_INPUT placeholders
            match_user = re.fullmatch(r'\[USER_INPUT_([a-zA-Z0-9_]+)\]', template_obj)
            if match_user:
                input_key = match_user.group(1)
                return self._get_with_default(user_inputs, f'USER_INPUT_{input_key}', f'User input {input_key} not provided.')
            
            # Handle simple calculations (extend this as needed)
            if template_obj == "[CALCULATED_NetDebt]":
                try:
                    gross_debt = float(self._get_with_default(user_inputs, 'USER_INPUT_GrossDebt_Current', 0))
                    cash = float(self._get_with_default(user_inputs, 'USER_INPUT_Cash_Current', 0))
                    return str(gross_debt - cash)
                except ValueError:
                    return "Error calculating Net Debt"
            if template_obj == "[CALCULATED_FCF_EBITDA_Percent]":
                 try:
                    fcf = float(self._get_with_default(user_inputs, 'USER_INPUT_FCF_CurrentYear', 0))
                    ebitda = float(self._get_with_default(user_inputs, 'USER_INPUT_EBITDA_CurrentYear', 0))
                    return f"{(fcf / ebitda * 100):.1f}" if ebitda else "N/A"
                 except ValueError:
                    return "Error calculating FCF/EBITDA"
            if template_obj == "[CALCULATED_CumulativeFCF_OverPeriod]":
                try:
                    fcf = float(self._get_with_default(user_inputs, 'USER_INPUT_FCF_CurrentYear', 0))
                    years = int(self._get_with_default(user_inputs, 'USER_INPUT_DebtRepayment_ForecastYears', 1))
                    return str(fcf * years) # Simplified assumption
                except ValueError:
                    return "Error calculating Cumulative FCF"
            if template_obj == "[CALCULATED_CumulativeFCF_to_Debt_Percent]":
                try:
                    cum_fcf = float(self._populate_json_ld_recursively("[CALCULATED_CumulativeFCF_OverPeriod]", user_inputs))
                    debt = float(self._get_with_default(user_inputs, 'USER_INPUT_GrossDebt_Current', 0))
                    return f"{(cum_fcf / debt * 100):.1f}" if debt else "N/A"
                except ValueError:
                    return "Error calculating FCF/Debt %"

            # Handle LLM/CACM placeholders (simulated)
            if template_obj.startswith("[LLM_CACM_Generate") or template_obj.startswith("[LLM_Analyze") or \ 
               template_obj.startswith("[LLM_CACM_Assess") or template_obj.startswith("[LLM_CACM_Describe") or \ 
               template_obj.startswith("[LLM_CACM_Identify") or template_obj.startswith("[LLM_CACM_Note") or \ 
               template_obj.startswith("[LLM_CACM_Provide") or template_obj.startswith("[LLM_CACM_Summarize") or \ 
               template_obj.startswith("[LLM_CACM_List") or template_obj.startswith("[LLM_CACM_Estimate") or \ 
               template_obj.startswith("[LLM_CACM_Project") or template_obj.startswith("[LLM_CACM_Calculate") or \ 
               template_obj.startswith("[LLM_Identify") or template_obj.startswith("[LLM_Determine") or \ 
               template_obj.startswith("[LLM_Synthesize") or template_obj.startswith("[LLM_Retrieve") or \ 
               template_obj.startswith("[LLM_Compare") or template_obj.startswith("[LLM_Explain"):
                # Simple simulation: return a descriptive string based on the placeholder
                action = template_obj.strip('[]').replace("LLM_CACM_", "Simulated: ").replace("LLM_", "Simulated: ").replace("_"," ")
                # Add some context from user inputs if possible
                company_name = self._get_with_default(user_inputs, 'USER_INPUT_CompanyName', 'the company')
                if 'BusinessDescription' in action: return f"Detailed business description for {company_name} focusing on core operations, competitive advantages, and strategy (simulated)."
                if 'ProductsServices' in action: return f"Overview of key products/services offered by {company_name}, their market segments, and revenue contribution (simulated)."
                if 'OperatingGeographies' in action: return f"Main operating geographies for {company_name} and revenue breakdown by region if available (simulated)."
                if 'ScaleMarketPosition' in action: return f"Assessment of {company_name}'s scale (revenue, employees, assets) and market share/position vs peers (simulated)."
                if 'CorporateStructure' in action: return f"Description of {company_name}'s corporate structure, including key subsidiaries and parent relationships if any (simulated)."
                if 'SummarizeNews' in action: return f"Summary of recent significant news for {company_name} based on user input and simulated search, highlighting credit-relevant events (simulated)."
                if 'SummarizeLatestEarnings' in action: return f"Summary of {company_name}'s latest earnings call, key financial highlights, and management outlook (simulated)."
                if 'CreditImpact Of Events' in action: return f"Analysis of potential credit impact (positive/negative/neutral) of recent developments for {company_name} (simulated)."
                if 'Simulated: Identify IndustryTrends' in action: return f"Key trends in the {self._get_with_default(user_inputs, 'USER_INPUT_PrimarySector', 'relevant')} sector, e.g., technological disruption, regulatory changes (simulated)."
                if 'Simulated: Assess CompanyPosition In Industry' in action: return f"{company_name}'s competitive positioning within its industry, including strengths and weaknesses relative to peers mentioned (simulated)."
                if 'FinancialDataSources Adjustments' in action: return "Financials sourced from user inputs. Standard accounting principles assumed. No material adjustments noted unless specified by user (simulated)."
                if 'Simulated: Analyze RevenueTrend 3-5Years' in action: return f"Revenue trend analysis for {company_name} over the past 3-5 years (simulated based on available data - if only one year provided, trend is flat)."
                if 'Simulated: Determine InternalCreditRating' in action: return self._infer_rating_outlook(user_inputs)[0]
                if 'Simulated: Determine RatingOutlook' in action: return self._infer_rating_outlook(user_inputs)[1]
                if 'Simulated: Generate DetailedRatingRationale' in action: return f"The rating for {company_name} is based on [simulated synthesis of financial strength, market position, and qualitative factors like management and industry outlook, offset by identified risks]."
                if 'Simulated: List PositiveRatingFactors' in action: return "- Strong FCF Generation\n- Solid Market Share\n- Experienced Management (simulated list based on positive inputs)"
                if 'Simulated: List NegativeRatingFactors' in action: return "- High Leverage (if applicable)\n- Competitive Pressures\n- Regulatory Headwinds (simulated list based on potential negative inputs)"
                if 'Simulated: Identify KeyDowngradeTriggers' in action: return "- Significant deterioration in profitability or FCF\n- Sustained increase in leverage above X.Xx\n- Loss of key market share (simulated list)"
                if 'Simulated: Synthesize Strength' in action: return f"{action} (simulated strength for {company_name}) based on inputs like financial performance or market position."
                if 'Simulated: Synthesize Weakness' in action: return f"{action} (simulated weakness for {company_name}) based on inputs like high debt or competitive concerns."
                return f"{action} for {company_name} (simulated)."
            
            if template_obj == "[CURRENT_DATE]":
                return datetime.date.today().isoformat()
            
            # For other placeholders not matched, return them as is or with a note
            if template_obj.startswith("[") and template_obj.endswith("]"):
                 return f"Data for '{template_obj}' to be determined or input by user/system (simulated)."
            return template_obj # Return non-placeholder strings as is
        return template_obj # Return other types as is

    def _infer_rating_outlook(self, inputs_dict):
        # Simplified rating logic for simulation 
        score = 0
        try:
            if float(self._get_with_default(inputs_dict, 'USER_INPUT_FCF_CurrentYear', 0)) > 50: score += 2
            elif float(self._get_with_default(inputs_dict, 'USER_INPUT_FCF_CurrentYear', 0)) > 0: score += 1
            else: score -= 1

            debt_ebitda = float(self._get_with_default(inputs_dict, 'USER_INPUT_Debt_EBITDA_Ratio', 3))
            if debt_ebitda < 1.5: score += 2
            elif debt_ebitda < 3.0: score += 1
            elif debt_ebitda > 4.5: score -= 2
            else: score -= 1
            
            ebitda_interest = float(self._get_with_default(inputs_dict, 'USER_INPUT_EBITDA_Interest_Ratio', 5))
            if ebitda_interest > 8.0: score += 2
            elif ebitda_interest > 4.0: score += 1
            elif ebitda_interest < 2.0: score -= 2
            else: score -= 1
            
            if "strong execution" in self._get_with_default(inputs_dict, 'USER_INPUT_ManagementStrategy', '').lower(): score +=1
            if "intense competition" in self._get_with_default(inputs_dict, 'USER_INPUT_CompetitiveLandscape', '').lower(): score -=1
        except:
            pass # Ignore conversion errors for this simple simulation

        if score >= 5: return ("A", "Positive")
        if score >= 3: return ("A-", "Stable")
        if score >= 1: return ("BBB+", "Stable")
        if score >= -1: return ("BBB", "Stable")
        if score >= -3: return ("BBB-", "Negative")
        if score >= -5: return ("BB+", "Negative")
        return ("BB", "Negative")

    def generate_populated_json_ld(self, user_inputs):
        # Deep copy template to avoid modifying it in-place
        import copy
        populated_template = copy.deepcopy(self.master_template)
        return self._populate_json_ld_recursively(populated_template, user_inputs)

    def generate_markdown_report(self, populated_json_ld):
        # Extract data from populated JSON-LD to fill the Markdown template
        # This is a simplified extraction; a real version would navigate the JSON-LD more robustly
        data = {}
        try:
            company_info = populated_json_ld.get('targetCompanyInformation', {})
            data['company_name'] = company_info.get('companyName', 'N/A')
            data['company_ticker'] = company_info.get('tickerSymbol', 'N/A')
            data['company_sector'] = company_info.get('primarySector', 'N/A')
            data['company_industry'] = company_info.get('primaryIndustry', 'N/A')
            data['company_country'] = company_info.get('countryOfDomicile', 'N/A')
            data['analyst_key_assumptions'] = str(company_info.get('analystKeyAssumptions', 'N/A')).replace('\n', '\n    * ')

            sections = populated_json_ld.get('reportSections', {})
            co_overview = sections.get('companyOverview', {})
            data['company_business_description'] = co_overview.get('businessDescription', 'N/A')
            data['company_products_services'] = co_overview.get('productsAndServices', 'N/A')
            data['company_operating_geographies'] = co_overview.get('operatingGeographies', 'N/A')
            data['company_scale_market_position'] = co_overview.get('scaleAndMarketPosition', 'N/A')
            data['company_corporate_structure'] = co_overview.get('corporateStructure', 'N/A')

            fin_analysis = sections.get('financialPerformanceAnalysis', {})
            data['financial_analysis_period'] = fin_analysis.get('analysisPeriod', 'N/A')
            data['financial_data_sources_notes'] = fin_analysis.get('dataSourceNotes', 'N/A')
            rev_analysis = fin_analysis.get('revenueAnalysis', {})
            data['financial_revenue_total'] = rev_analysis.get('totalRevenue', {}).get('value', 'N/A')
            data['financial_revenue_growth_yoy'] = rev_analysis.get('revenueGrowthYoY', {}).get('value', 'N/A')
            data['financial_revenue_trend'] = rev_analysis.get('revenueTrend', 'N/A')
            data['financial_revenue_drivers'] = rev_analysis.get('revenueDrivers', 'N/A')
            data['financial_revenue_commentary'] = rev_analysis.get('revenueCommentary', 'N/A')
            
            prof_analysis = fin_analysis.get('profitabilityAnalysis', {})
            data['financial_ebitda_total'] = prof_analysis.get('ebitda', {}).get('value', 'N/A')
            data['financial_ebitda_margin'] = prof_analysis.get('ebitdaMargin', {}).get('value', 'N/A')
            data['financial_ebitda_trend'] = prof_analysis.get('ebitdaTrend', 'N/A')
            data['financial_ebitda_margin_trend'] = prof_analysis.get('ebitdaMarginTrend', 'N/A')
            data['financial_profitability_drivers'] = prof_analysis.get('profitabilityDrivers', 'N/A')
            data['financial_profitability_commentary'] = prof_analysis.get('profitabilityCommentary', 'N/A')

            cf_analysis = fin_analysis.get('cashFlowAnalysis', {})
            data['financial_ocf'] = cf_analysis.get('operatingCashFlow',{}).get('value', 'N/A')
            data['financial_capex'] = cf_analysis.get('capitalExpenditures',{}).get('value', 'N/A')
            data['financial_fcf'] = cf_analysis.get('freeCashFlow',{}).get('value', 'N/A')
            data['financial_fcf_trend'] = cf_analysis.get('fcfTrend','N/A')
            data['financial_fcf_conversion'] = cf_analysis.get('fcfConversionRate',{}).get('value','N/A')
            data['financial_cashflow_drivers'] = cf_analysis.get('cashFlowDrivers','N/A')
            data['financial_cashflow_commentary'] = cf_analysis.get('cashFlowCommentary','N/A')

            lev_analysis = fin_analysis.get('leverageAnalysis', {})
            data['financial_gross_debt'] = lev_analysis.get('grossDebt',{}).get('value', 'N/A')
            data['financial_cash'] = lev_analysis.get('cashAndEquivalents',{}).get('value', 'N/A')
            data['financial_net_debt'] = lev_analysis.get('netDebt',{}).get('value', 'N/A')
            data['financial_debt_ebitda_ratio'] = lev_analysis.get('grossDebtToEbitda',{}).get('value', 'N/A')
            data['financial_leverage_trend'] = lev_analysis.get('leverageTrend','N/A')
            data['financial_leverage_drivers'] = lev_analysis.get('leverageDrivers','N/A')
            data['financial_debt_structure'] = lev_analysis.get('debtStructureMaturityProfile','N/A')
            data['financial_leverage_commentary'] = lev_analysis.get('leverageCommentary','N/A')

            # ... (Continue extracting for all sections and sub-sections from populated_json_ld)
            # This will be extensive to map everything from your JSON-LD to the Markdown placeholders.
            # For brevity, I'll show a few more key ones for the Executive Summary and Rating.

            rating_section = sections.get('creditRatingAssessment', {})
            internal_rating = rating_section.get('internalCreditRating', {})
            data['exec_summary_simulated_rating'] = internal_rating.get('ratingValue', 'N/A')
            data['exec_summary_rating_outlook'] = internal_rating.get('ratingOutlook', 'N/A')
            data['rating_internal_value'] = internal_rating.get('ratingValue', 'N/A')
            data['rating_internal_scale'] = internal_rating.get('ratingScale', 'N/A')
            data['rating_internal_outlook'] = internal_rating.get('ratingOutlook', 'N/A')
            data['rating_date'] = internal_rating.get('ratingAssignedDate', datetime.date.today().isoformat())
            data['rating_rationale_detailed'] = internal_rating.get('ratingRationale', 'Rationale not simulated.')
            
            # Simplified Exec Summary parts (a real LLM would synthesize these)
            data['exec_summary_overall_assessment'] = f"Overall assessment for {data['company_name']} ({data['company_ticker']}) suggests a credit profile of {data['exec_summary_simulated_rating']} with a {data['exec_summary_rating_outlook']} outlook, based on current inputs and simulated analysis."
            data['exec_summary_key_positive_factors'] = "    * Simulated Strong FCF\n    * Simulated Moderate Leverage"
            data['exec_summary_key_credit_concerns'] = "    * Simulated Competitive Market\n    * Simulated Regulatory Uncertainty"
            
            # Fill in other placeholders with 'N/A' or simulated content if not extracted above
            for key in self.markdown_template.split('{'): # Get all placeholders
                if '}' in key:
                    placeholder = key.split('}')[0]
                    if placeholder not in data:
                        data[placeholder] = f"[Simulated content for {placeholder}]"
            data['report_date'] = datetime.date.today().isoformat()
            data['report_disclaimer'] = populated_json_ld.get('reportGenerationInstructions', {}).get('mandatoryDisclaimer', 'Standard Disclaimer.')

        except Exception as e:
            print(f"Error during Markdown report generation: {e}")
            # Fallback for data if extraction fails
            for key in self.markdown_template.split('{'):
                 if '}' in key:
                    placeholder = key.split('}')[0]
                    if placeholder not in data:
                        data[placeholder] = f"[Error extracting: {placeholder}]"
            data['company_name'] = populated_json_ld.get('targetCompanyInformation',{}).get('companyName', 'Error')
            data['company_ticker'] = populated_json_ld.get('targetCompanyInformation',{}).get('tickerSymbol', 'Error')
            data['report_date'] = datetime.date.today().isoformat()
            data['report_disclaimer'] = "Error in report generation - data incomplete."

        return self.markdown_template.format(**data)

report_generator = ReportGeneratorSimulator(MASTER_JSON_LD_TEMPLATE, REPORT_STRUCTURE_GUIDE_MD)

# --- UI Generation (from input_ui_config.json structure) ---
input_widgets = {}
sections_vbox_list = []

for section_idx, section in enumerate(INPUT_UI_CONFIG['sections']):
    section_title_html = f"<h3 style='margin-top: 15px; margin-bottom: 5px; border-top: 1px solid #ccc; padding-top: 10px;'>{section_idx+1}. {section['title']}</h3>"
    if section_idx == 0: section_title_html = f"<h3 style='margin-bottom: 5px;'>{section_idx+1}. {section['title']}</h3>"
    section_title_widget = widgets.HTML(section_title_html)
    fields_widgets_list = [section_title_widget]
    for field_config in section['fields']:
        label_widget = widgets.Label(field_config['label'], layout=widgets.Layout(width='250px'))
        if field_config['type'] == 'text':
            widget = widgets.Text(value=str(field_config['default']), layout=widgets.Layout(width='70%'))
        elif field_config['type'] == 'textarea':
            widget = widgets.Textarea(value=str(field_config['default']), layout=widgets.Layout(width='70%', height='80px'))
        elif field_config['type'] == 'float':
            widget = widgets.FloatText(value=float(field_config['default']), layout=widgets.Layout(width='150px'))
        elif field_config['type'] == 'int':
            widget = widgets.IntText(value=int(field_config['default']), layout=widgets.Layout(width='150px'))
        else:
            widget = widgets.Text(value=str(field_config['default']), layout=widgets.Layout(width='70%'))
        
        input_widgets[field_config['name']] = widget
        fields_widgets_list.append(widgets.HBox([label_widget, widget], layout=widgets.Layout(margin_bottom='2px')))
    sections_vbox_list.append(widgets.VBox(fields_widgets_list))

all_inputs_vbox = widgets.VBox(sections_vbox_list)

# --- Button and Output Areas ---
generate_button = widgets.Button(
    description="Generate Populated JSON-LD & Simulated Report",
    button_style='success',
    layout=widgets.Layout(width='auto', margin_top='20px', margin_bottom='10px')
)
json_ld_output_area = widgets.Output()
markdown_report_output_area = widgets.Output()

def on_generate_button_clicked(b):
    user_inputs_collected = {name: w.value for name, w in input_widgets.items()}
    
    with json_ld_output_area:
        json_ld_output_area.clear_output(wait=True)
        display(HTML("<h4>Processing JSON-LD...</h4>"))
        populated_json_ld = report_generator.generate_populated_json_ld(user_inputs_collected)
        display(HTML("<h2>Populated JSON-LD Prompt (for review/saving):</h2>"))
        # Use Javascript to pretty print JSON in a collapsible way if possible, else plain text
        json_str_pretty = json.dumps(populated_json_ld, indent=2)
        display(HTML(f"<pre style='white-space: pre-wrap; background-color: #f0f0f0; padding: 10px; border: 1px solid #ccc; max-height: 400px; overflow-y: auto;'>{json_str_pretty}</pre>"))

    with markdown_report_output_area:
        markdown_report_output_area.clear_output(wait=True)
        display(HTML("<h4>Generating Markdown Report...</h4>"))
        # Pass the already populated JSON-LD to the markdown generator part
        simulated_report_md = report_generator.generate_markdown_report(populated_json_ld) 
        display(HTML("<h2 style='margin-top: 20px;'>Simulated Human-Readable Credit Report:</h2>"))
        display(Markdown(simulated_report_md))

generate_button.on_click(on_generate_button_clicked)

display(HTML("<h1>Interactive Credit Report Tool (JSON-LD Core)</h1>"))
display(all_inputs_vbox)
display(generate_button)
display(HTML("<hr>"))
display(json_ld_output_area)
display(HTML("<hr>"))
display(markdown_report_output_area)
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.12" 
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
